In [16]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
import requests
from bs4 import BeautifulSoup
import os
import ollama
import json
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from pinecone import Pinecone
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import json
import time
import pandas as pd
from openai import OpenAI
import ollama
import datetime
from jinja2 import Template
import os
from openai import OpenAI
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

PINECONE_HOST_URL = os.getenv("PINECONE_HOST_URL")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
HOME_URL = os.getenv("HOME_URL")
SERVICES_URL = os.getenv("SERVICES_URL")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
ENDPOINT = os.getenv("ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
DEPLOYMENT_NAME = os.getenv("DEPLOYMENT_NAME")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(host = PINECONE_HOST_URL)

client = OpenAI(api_key=OPENAI_API_KEY)
EMBEDDINGS = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-exp-03-07", google_api_key=GOOGLE_API_KEY)


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}


# Parse the HTML content


def get_openai_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def get_gemini_embedding(chunk_data):
    vector = EMBEDDINGS.embed_query(chunk_data, output_dimensionality = 1536)
    return vector


with open("prompt_template.md", "r", encoding="utf-8") as f:
    template_text = f.read()

# Create Jinja2 template
template = Template(template_text)

#get partners of the company


#get testimonials
def search_testimonials(search_vector):
    match_results = index.query(
        namespace="testimonials",
        vector=search_vector, 
        top_k=2,
        include_metadata=True,
        include_values=False
    )
    print("\Testimonials retrieval successful")
    return match_results

#get certifications
def search_certifications(search_vector):
    match_results = index.query(
        namespace="certifications",
        vector=search_vector, 
        top_k=15,
        include_metadata=True,
        include_values=False
    )
    print("\nCertifications retrieval successful")
    return match_results

#get services offered by the company
def get_services_provided(URL):
    response = requests.get(URL, headers=headers)
    #service_response = requests.get(os.getenv("SERVICES_URL"), headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    #service_soup = BeautifulSoup(service_response.text, 'html.parser')
    services_content = soup.find_all('h2', class_ = "elementor-heading-title elementor-size-default")
    print(len(services_content))

    services_provided = []
    for tag in services_content:
        services_provided.append(tag.get_text(strip=True))
    return services_provided  

#------------------------------------------------------------------------

endpoint = os.getenv("ENDPOINT_URL", "https://opal-sales-dev-ai.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4.1")
      
# Initialize Azure OpenAI client with Entra ID authentication

client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_version="2025-01-01-preview",
)

def get_prompt_response(user_input, service_url, client):
    final_prompt = template.render(
        user_input=user_input,
        services=get_services_provided(service_url),
        testimonials=search_testimonials(get_gemini_embedding(user_input)),
        certifications=search_certifications(get_gemini_embedding(user_input))
    )
    response = client.chat.completions.create(  
        model = 'gpt-4.1', 
        temperature= 0.6,
        frequency_penalty=0,
        presence_penalty=0,
        messages=[
            {"role": "system", "content":final_prompt},

      ]
    )
    content = response.choices[0].message.content.strip()

    #call the response model
    return content


<>:77: SyntaxWarning: invalid escape sequence '\T'
<>:77: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Admin\AppData\Local\Temp\ipykernel_21396\4152170786.py:77: SyntaxWarning: invalid escape sequence '\T'
  print("\Testimonials retrieval successful")


In [31]:
text = "How Hibiz contributed and showcased its expertise in Salesforce ecosystem"

In [32]:
print(get_prompt_response(text, HOME_URL, client))


9
\Testimonials retrieval successful

Certifications retrieval successful
```markdown
# Hibiz Solutions: Showcasing Expertise in the Salesforce Ecosystem

## 🚀 Introduction

In today’s rapidly evolving digital landscape, leveraging the right CRM platform is pivotal for business success. Hibiz Solutions stands at the forefront of Salesforce ecosystem consulting and implementation, empowering organizations to transform their customer engagement, sales processes, and operational efficiency.

---

## 🌟 Our Salesforce Expertise: Transforming Vision into Value

### 1. **Comprehensive Salesforce Services**

Hibiz Solutions provides end-to-end Salesforce services, including:

| Service Area                      | Description                                                                                  |
|------------------------------------|----------------------------------------------------------------------------------------------|
| Salesforce Implementation         | Custom deployment 

In [27]:
%pip install azure-identity

     ---------------------------------------- 0.0/81.9 kB ? eta -:--:--
     ----- ---------------------------------- 10.2/81.9 kB ? eta -:--:--
     -------------- ----------------------- 30.7/81.9 kB 445.2 kB/s eta 0:00:01
     -------------------------------------- 81.9/81.9 kB 770.1 kB/s eta 0:00:00
  Using cached msal-1.32.3-py3-none-any.whl.metadata (11 kB)
   ---------------------------------------- 0.0/186.1 kB ? eta -:--:--
   ---------------------------------------- 186.1/186.1 kB 3.7 MB/s eta 0:00:00
Using cached msal-1.32.3-py3-none-any.whl (115 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
import os
import base64
from openai import AzureOpenAI
      
endpoint = os.getenv("ENDPOINT_URL", "https://opal-sales-dev-ai.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4.1")
      
# Initialize Azure OpenAI client with Entra ID authentication

client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_version="2025-01-01-preview",
)


# IMAGE_PATH = "YOUR_IMAGE_PATH"
# encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')
chat_prompt = [
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "You are an AI assistant that helps people find information."
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "hi how are you"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "Hello! I’m here and ready to help you. How can I assist you today?"
            }
        ]
    }
]

# Include speech result if speech is enabled
messages = chat_prompt

completion = client.chat.completions.create(
    model=deployment,
    messages=messages,
    max_tokens=800,
    temperature=1,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False
)

print(completion.to_json())

{
  "id": "chatcmpl-Bs8DZB5g88FTgSBzcsZHKUAliUyWh",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello! I'm just a program, but thanks for asking. How can I help you today?",
        "refusal": null,
        "role": "assistant",
        "annotations": []
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1752241001,
  "model": "gpt-4.1-2025-04-14",
  "object": "chat.completion",
  "system_fingerprint": "fp_07e970ab25",
  "usage": {
    "completion_tokens": 20,
    "prompt_tokens": 48,
    "total_tokens": 68,
    "completion_